# CS475 Natural Language Model - Programming Assignment 1

## Submission Guide

**How to submit**
* Fill out <mark>TODO</mark> blocks, **DO NOT** modify other parts of the skeleton code.
* Submit one file: hw1_{student_ID}.ipynb to KLMS
    e.g.hw1_20243150.ipynb
* **Late submission policy**: After the submission deadline, you will
immediately lose 10% of the score, another 10% after 24 hours later, and so on.
Submission after 72 hours (3 days) will not be counted.

**Note**
* Make a copy of this .ipynb file. Do not directly edit this file.
* You are required to use numpy, do not use neither pytorch nor tensorflow.
* Check whether your whole cells work well by restarting runtime code and running all before the submission.
* TA will look into the implemented functions, their validity and give corresponding score to each <mark>TODO</mark> problem.
* Ask questions through KLMS so that you can share information with other students.
* TA in charge: Yeahoon Kim (yeahoon.kim@kaist.ac.kr)

**In this programming assignment, you will**
* Learn how the backporpagation works.
* Learn how the pytorch module works.

In [ ]:
# Import cell
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Derivation of a simple function with one input

### 1.1. What is derivation?

Derivation is a mathematical process used to find the rate of change of a function with respect to its input variables. It is an essential tool in calculus and is widely used in various fields of science and engineering.

In programming, we can approximate the derivative of a function by using numerical methods. One common method is called the **"finite difference approximation."**

The finite difference approximation calculates the derivative by estimating the slope of the function at a given point. It does this by taking the difference between the function values at two nearby points and dividing it by the difference in their corresponding input values. This can be expressed as an equation as follows:

$\frac{f(x+h)-f(x)}{h}$

You can check the detail in [here](https://en.wikipedia.org/wiki/Finite_difference)

### 1.2. Simple example of the derivation
The following is the basic example of the derivation

In [ ]:
################## TODO: Implement the gradient function ##################
# The gradient function should take a function f and a point x and return the gradient of f at x.
# The gradient should be approximated using finite differences with a small step size h.
# The function f is assumed to be scalar-valued.
def gradient(f, x: float, h=1e-5) -> float:
    
    
    
    
    return 0
###########################################################################

In [ ]:
# Define function f(x)
def f(x: float) -> float:
    return 3 * x**2 - 4 * x + 5

# Find the gradient of f(x) at x = 2
h = 0.00001
x = 2
df = gradient(f, x, h)

print(f"Gradient of function f at the point {x}: {df}")

In [ ]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)
plt.plot(xs, ys)

## 2. Implement backpropagation

### 2.1. Implement value object for the gradient flow
The `Value` object is a class that represents a scalar value and its gradient. It is used in the context of gradient descent and backpropagation algorithms. The `Value` object has the following attributes and methods:

Attributes:
- `data`: The scalar value stored in the object.
- `grad`: The gradient of the scalar value with respect to some variable.
- `_backward`: A function that defines the backward pass of the backpropagation algorithm.
- `_prev`: A set of `Value` objects that are the inputs to the operation that produced this object.
- `_op`: A string that represents the operation that produced this object.

Methods:
- `__add__(self, other)`: Overloads the `+` operator to perform addition between two `Value` objects or a `Value` object and a scalar.
- `__mul__(self, other)`: Overloads the `*` operator to perform multiplication between two `Value` objects or a `Value` object and a scalar.
- `__pow__(self, other)`: Overloads the `**` operator to perform exponentiation between a `Value` object and an integer or float.
- `relu(self)`: Applies the rectified linear unit (ReLU) activation function to the `Value` object.
- `backward(self)`: Performs the backward pass of the backpropagation algorithm to compute the gradients of the `Value` object and its inputs.

The `Value` object is used in the implementation of neural networks and other machine learning algorithms to compute gradients and update the model parameters during training. It allows for automatic differentiation and efficient computation of gradients using the chain rule.

<mark>TODO</mark> Implement the activation functions.

In [ ]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    ############# TODO: implement more activation functions ############
    # You can implement more activations such as sigmoid, tanh, etc.
    # Make sure to implement their gradients as well.
    
    def sigmoid(self):
        # You can refer to the following link to implement the sigmoid function
        # https://en.wikipedia.org/wiki/Sigmoid_function
        pass
    
    def tanh(self):
        # You can refer to the following link to implement the tanh function
        # https://en.wikipedia.org/wiki/Hyperbolic_function
        pass
    
    def SiLU(self):
        # You can refer to the following link to implement the SiLU function
        # https://arxiv.org/abs/1702.03118
        pass
    
    def GELU(self):
        # You can refer to the following link to implement the GELU function
        # https://arxiv.org/abs/1606.08415
        pass
    
    def ELU(self):
        # You can refer to the following link to implement the ELU function
        # https://arxiv.org/abs/1511.07289
        pass
    
    ######################################################################
    
    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"


In [ ]:
a = Value(-4.0)
b = Value(2.0)
c = a + b
d = a * b + b**3
c += c + 1
c += 1 + c + (-a)
d += d * 2 + (b + a).relu()
d += 3 * d + (b - a).relu()
e = c - d
f = e**2
g = f / 2.0
g += 10.0 / f

print(f'{g.data:.4f}') # prints 24.7041, the outcome of this forward pass
g.backward()
print(f'{a.grad:.4f}') # prints 138.8338, i.e. the numerical value of dg/da
print(f'{b.grad:.4f}') # prints 645.5773, i.e. the numerical value of dg/db

# 3. Implement Neural Networks

In [ ]:
class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

In [ ]:
class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.relu() if self.nonlin else act

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

In [ ]:
class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

In [ ]:
class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"

# 4. Use Neural Networks in MNIST